In [17]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import numpy as np

# โหลดตัวแปรจาก .env
load_dotenv()

# ดึงค่าจาก environment
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')  
database = 'fininsurance'

# สร้าง engine สำหรับเชื่อมต่อฐานข้อมูล
engine = create_engine(f'mariadb+mariadbconnector://{user}:{password}@{host}:{port}/{database}')

# SQL query
query = """
SELECT cuscode, name,
       CASE 
       WHEN user_registered = '0000-00-00 00:00:00.000' THEN '2000-01-01 00:00:00'
         ELSE user_registered 
       END AS user_registered,
       idcard, card_ins,card_ins_id,card_ins_type,card_ins_life,card_ins_type_life,file_card_ins_life,card_ins_start,card_ins_exp,
       card_ins_life_exp,is_move_card_ins
FROM wp_users WHERE user_login NOT IN ('FINTEST-01', 'FIN-TestApp', 'Admin-VIF', 'adminmag_fin', 'FNG00-00001')


"""


# โหลดข้อมูลจากฐานข้อมูล
df_merged = pd.read_sql(query, engine)

# แปลงให้ pandas เข้าใจได้แน่นอน
df_merged['user_registered'] = pd.to_datetime(df_merged['user_registered'].astype(str), errors='coerce')
df_merged


,cuscode,name,user_registered,idcard,card_ins,card_ins_id,card_ins_type,card_ins_life,card_ins_type_life,file_card_ins_life,card_ins_start,card_ins_exp,card_ins_life_exp,is_move_card_ins
0,FINBANK,None,2019-07-12 07:13:34,None,None,None,None,None,None,None,None,None,None,2.0
1,punyawat@dplusonline.net,None,2019-08-19 04:30:13,None,None,None,None,None,None,None,None,None,None,2.0
2,FIN19-jay,None,2019-11-25 06:35:16,None,None,None,None,None,None,None,None,None,None,2.0
3,FIN0000000,ปวีณา ชมสวน,2019-11-28 02:53:39,None,None,None,None,None,None,None,None,None,None,2.0
4,FNG1911-0037,ปัญญา,2019-11-28 06:34:37,1101500574600,yes,None,None,None,None,None,None,None,None,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140607,FNG25-139208,มนฑาทิพย์ ใจรังษี,2025-07-11 13:26:25,3801500174197,yes,None,A,yes,B,None,None,None,None,NaN
140608,FNG25-139209,เศรษฐาพินิจกุล,2025-07-11 13:33:35,1399900049905,no,None,N,no,N,https://finbrokerfile.sgp1.digitaloceanspaces....,None,None,None,NaN
140609,FNG25-139210,รสสุคนธ์ ท้าวจันทร์,2025-07-11 13:38:29,None,None,None,None,None,None,None,None,None,None,NaN
140610,FNG25-139211,ปิยรัฐ ปรียากร,2025-07-11 13:42:37,3840100378096,None,None,None,None,None,None,None,None,None,NaN


In [18]:
df_merged = df_merged.drop(columns=['user_registered'])

In [19]:
for col in df_merged.columns:
    print(col)


cuscode
name
idcard
card_ins
card_ins_id
card_ins_type
card_ins_life
card_ins_type_life
file_card_ins_life
card_ins_start
card_ins_exp
card_ins_life_exp
is_move_card_ins


In [57]:
rename_columns = {
    "cuscode": "agent_id",
    "name": "agent_name",
    "idcard": "id_card",
    "card_ins": "card_ins",
    "card_ins_id": "card_ins_id",
    "card_ins_type": "card_ins_type",
    "card_ins_life": "card_ins_life",
    "card_ins_type_life": "card_ins_type_life",
    "file_card_ins_life": "file_card_ins_life",
    "card_ins_start": "card_ins_start",
    "card_ins_exp": "card_ins_exp",
    "card_ins_life_exp": "card_ins_life_exp",
    "is_move_card_ins": "is_move_card_ins"

}

df = df_merged.rename(columns=rename_columns)
df

,agent_id,agent_name,id_card,card_ins,card_ins_id,card_ins_type,card_ins_life,card_ins_type_life,file_card_ins_life,card_ins_start,card_ins_exp,card_ins_life_exp,is_move_card_ins
0,FINBANK,None,None,None,None,None,None,None,None,None,None,None,2.0
1,punyawat@dplusonline.net,None,None,None,None,None,None,None,None,None,None,None,2.0
2,FIN19-jay,None,None,None,None,None,None,None,None,None,None,None,2.0
3,FIN0000000,ปวีณา ชมสวน,None,None,None,None,None,None,None,None,None,None,2.0
4,FNG1911-0037,ปัญญา,1101500574600,yes,None,None,None,None,None,None,None,None,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140607,FNG25-139208,มนฑาทิพย์ ใจรังษี,3801500174197,yes,None,A,yes,B,None,None,None,None,NaN
140608,FNG25-139209,เศรษฐาพินิจกุล,1399900049905,no,None,N,no,N,https://finbrokerfile.sgp1.digitaloceanspaces....,None,None,None,NaN
140609,FNG25-139210,รสสุคนธ์ ท้าวจันทร์,None,None,None,None,None,None,None,None,None,None,NaN
140610,FNG25-139211,ปิยรัฐ ปรียากร,3840100378096,None,None,None,None,None,None,None,None,None,NaN


In [58]:
for col in df.columns:
    print(col)

agent_id
agent_name
id_card
card_ins
card_ins_id
card_ins_type
card_ins_life
card_ins_type_life
file_card_ins_life
card_ins_start
card_ins_exp
card_ins_life_exp
is_move_card_ins


In [59]:
df['card_ins_exp'].unique()


array([None, '', '2027-10-28', ..., '2025-12-19', '2026-05-16',
       '2025-07-21'], shape=(2765,), dtype=object)

In [61]:
import pandas as pd

def convert_thai_to_ad(date_str):
    try:
        if pd.isna(date_str) or date_str == '':
            return None
        day, month, year = map(int, date_str.split('-'))
        if year > 2500:
            year -= 543
        return f"{year:04d}-{month:02d}-{day:02d}"
    except:
        return None

# แปลงทั้งสองคอลัมน์
df['card_ins_start'] = df['card_ins_start'].apply(convert_thai_to_ad)

In [62]:
df['card_ins_start'] = pd.to_datetime(df['card_ins_start'], errors='coerce')


In [63]:
import pandas as pd

def convert_thai_to_ad(date_str):
    try:
        if pd.isna(date_str) or date_str == '':
            return None
        day, month, year = map(int, date_str.split('-'))
        if year > 2500:
            year -= 543
        return f"{year:04d}-{month:02d}-{day:02d}"
    except:
        return None

# ใช้กับคอลัมน์ card_ins_exp
df['card_ins_exp'] = df['card_ins_exp'].apply(convert_thai_to_ad)

# แปลงเป็น datetime ถ้าต้องการ
df['card_ins_exp'] = pd.to_datetime(df['card_ins_exp'], errors='coerce')


C:\Users\jakkapad.wan\AppData\Local\Temp\ipykernel_7716\2395833531.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['card_ins_exp'] = pd.to_datetime(df['card_ins_exp'], errors='coerce')


In [17]:
df['card_ins_type_life'] = df['card_ins_type_life'].apply(
    lambda x: 'B' if isinstance(x, str) and 'แทน' in x else x
)


In [64]:
df = df.replace(r'^\s*$', pd.NA, regex=True)  
df = df[df.count(axis=1) > 1]
df

,agent_id,agent_name,id_card,card_ins,card_ins_id,card_ins_type,card_ins_life,card_ins_type_life,file_card_ins_life,card_ins_start,card_ins_exp,card_ins_life_exp,is_move_card_ins
0,FINBANK,None,None,None,None,None,None,None,None,NaT,NaT,None,2.0
1,punyawat@dplusonline.net,None,None,None,None,None,None,None,None,NaT,NaT,None,2.0
2,FIN19-jay,None,None,None,None,None,None,None,None,NaT,NaT,None,2.0
3,FIN0000000,ปวีณา ชมสวน,None,None,None,None,None,None,None,NaT,NaT,None,2.0
4,FNG1911-0037,ปัญญา,1101500574600,yes,None,None,None,None,None,NaT,NaT,None,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140607,FNG25-139208,มนฑาทิพย์ ใจรังษี,3801500174197,yes,None,A,yes,B,None,NaT,NaT,None,NaN
140608,FNG25-139209,เศรษฐาพินิจกุล,1399900049905,no,None,N,no,N,https://finbrokerfile.sgp1.digitaloceanspaces....,NaT,NaT,None,NaN
140609,FNG25-139210,รสสุคนธ์ ท้าวจันทร์,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140610,FNG25-139211,ปิยรัฐ ปรียากร,3840100378096,None,None,None,None,None,None,NaT,NaT,None,NaN


In [65]:
import pandas as pd
import numpy as np

# แปลงช่องว่างทุกแบบเป็น NaN ชั่วคราว เพื่อการนับข้อมูล
df_temp = df.replace(r'^\s*$', np.nan, regex=True)

# เพิ่มคอลัมน์ช่วยนับจำนวนข้อมูล (non-null)
df['non_empty_count'] = df_temp.notnull().sum(axis=1)

# >>>> ส่วนที่แก้ไขตรงนี้ <<<<
# ตรวจสอบ agent_id ที่ไม่ว่าง (ไม่ใช่ NaN และไม่ใช่ช่องว่าง)
valid_agent_id_mask = df['agent_id'].astype(str).str.strip().ne('') & df['agent_id'].notna()

# แยกกลุ่มที่ agent_id ไม่ว่างและ agent_id ว่าง
df_with_id = df[valid_agent_id_mask]
df_without_id = df[~valid_agent_id_mask]

# คัดแถวที่ agent_id ซ้ำ โดยเก็บแถวที่มีข้อมูลมากที่สุด
df_with_id_cleaned = df_with_id.sort_values('non_empty_count', ascending=False).drop_duplicates(subset='agent_id', keep='first')

# รวมกลับ
df_cleaned = pd.concat([df_with_id_cleaned, df_without_id], ignore_index=True)

# ลบคอลัมน์ช่วย
df_cleaned = df_cleaned.drop(columns=['non_empty_count'])
df_cleaned = df_cleaned.replace(
    to_replace=r'^\s*$|(?i:^none$)|^-$',  # << แก้ตรงนี้
    value=np.nan,
    regex=True
)


df_cleaned.columns = df_cleaned.columns.str.lower()
df_cleaned


,agent_id,agent_name,id_card,card_ins,card_ins_id,card_ins_type,card_ins_life,card_ins_type_life,file_card_ins_life,card_ins_start,card_ins_exp,card_ins_life_exp,is_move_card_ins
0,FNG23-092725,ณิชากร จันทร์รอด,1809800137077,yes,6604000243,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2023-06-01,2024-05-01,2024-01-04,1.0
1,FNG23-095043,เกศริน ยามะ,1501100073624,yes,6204045233,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2019-10-10,2027-09-10,2026-12-20,1.0
2,FNG23-094566,กุสุมา ศรีจุ้ย,1349700052694,yes,5502003168,B,yes,B,https://finbrokerfile.sgp1.digitaloceanspaces....,2012-11-09,2014-11-08,2023-05-09,1.0
3,FNG23-098990,โสภิดา เมืองละออง,1909900396138,yes,6604009637,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2023-06-19,2024-06-18,2024-06-08,1.0
4,FNG23-094309,เรณุกา ช่างเขียน,1570300123438,yes,6204039943,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2019-09-20,2027-09-19,2027-09-19,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140089,FNG25-133222,ภัทรานิษฐ์ ตั้งไพบูลย์วณิช,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140090,FNG25-133217,พีระพล โพธิราช,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140091,FNG25-139171,มนัส สุริยะพรชัยกุล,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140092,FNG25-139200,ธีรนัย แน่นอุดร,None,None,None,None,None,None,None,NaT,NaT,None,NaN


In [66]:
df_cleaned.replace(np.nan, "NaN").isin(["none", "-", "None"]).sum()
df_cleaned

,agent_id,agent_name,id_card,card_ins,card_ins_id,card_ins_type,card_ins_life,card_ins_type_life,file_card_ins_life,card_ins_start,card_ins_exp,card_ins_life_exp,is_move_card_ins
0,FNG23-092725,ณิชากร จันทร์รอด,1809800137077,yes,6604000243,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2023-06-01,2024-05-01,2024-01-04,1.0
1,FNG23-095043,เกศริน ยามะ,1501100073624,yes,6204045233,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2019-10-10,2027-09-10,2026-12-20,1.0
2,FNG23-094566,กุสุมา ศรีจุ้ย,1349700052694,yes,5502003168,B,yes,B,https://finbrokerfile.sgp1.digitaloceanspaces....,2012-11-09,2014-11-08,2023-05-09,1.0
3,FNG23-098990,โสภิดา เมืองละออง,1909900396138,yes,6604009637,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2023-06-19,2024-06-18,2024-06-08,1.0
4,FNG23-094309,เรณุกา ช่างเขียน,1570300123438,yes,6204039943,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2019-09-20,2027-09-19,2027-09-19,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140089,FNG25-133222,ภัทรานิษฐ์ ตั้งไพบูลย์วณิช,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140090,FNG25-133217,พีระพล โพธิราช,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140091,FNG25-139171,มนัส สุริยะพรชัยกุล,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140092,FNG25-139200,ธีรนัย แน่นอุดร,None,None,None,None,None,None,None,NaT,NaT,None,NaN


In [ ]:
# df.to_csv('dim_car3.csv', index=False)

In [67]:
df_cleaned = df_cleaned.replace(r'^\.$', np.nan, regex=True)
df_cleaned

,agent_id,agent_name,id_card,card_ins,card_ins_id,card_ins_type,card_ins_life,card_ins_type_life,file_card_ins_life,card_ins_start,card_ins_exp,card_ins_life_exp,is_move_card_ins
0,FNG23-092725,ณิชากร จันทร์รอด,1809800137077,yes,6604000243,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2023-06-01,2024-05-01,2024-01-04,1.0
1,FNG23-095043,เกศริน ยามะ,1501100073624,yes,6204045233,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2019-10-10,2027-09-10,2026-12-20,1.0
2,FNG23-094566,กุสุมา ศรีจุ้ย,1349700052694,yes,5502003168,B,yes,B,https://finbrokerfile.sgp1.digitaloceanspaces....,2012-11-09,2014-11-08,2023-05-09,1.0
3,FNG23-098990,โสภิดา เมืองละออง,1909900396138,yes,6604009637,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2023-06-19,2024-06-18,2024-06-08,1.0
4,FNG23-094309,เรณุกา ช่างเขียน,1570300123438,yes,6204039943,A,yes,A,https://finbrokerfile.sgp1.digitaloceanspaces....,2019-09-20,2027-09-19,2027-09-19,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140089,FNG25-133222,ภัทรานิษฐ์ ตั้งไพบูลย์วณิช,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140090,FNG25-133217,พีระพล โพธิราช,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140091,FNG25-139171,มนัส สุริยะพรชัยกุล,None,None,None,None,None,None,None,NaT,NaT,None,NaN
140092,FNG25-139200,ธีรนัย แน่นอุดร,None,None,None,None,None,None,None,NaT,NaT,None,NaN


In [77]:
for col in df_cleaned.columns:
    print(col)

agent_id
agent_name
id_card
card_ins
card_ins_id
card_ins_type
card_ins_life
card_ins_type_life
file_card_ins_life
card_ins_start
card_ins_exp
card_ins_life_exp
is_move_card_ins


In [ ]:
import numpy as np

# แปลง hire_date เป็น epoch
df_cleaned["hire_date"] = pd.to_datetime(df_cleaned["hire_date"], errors='coerce')
df_cleaned["hire_date"] = df_cleaned["hire_date"].astype('int64') // 10**9

# ถ้าต้องการให้ NaN เป็น None สำหรับ SQLAlchemy
df_cleaned["hire_date"] = df_cleaned["hire_date"].where(df_cleaned["hire_date"].notnull(), None)


In [84]:
# ถ้ามี NaN ให้เติมเป็น 0 ก่อน
df_cleaned['is_move_card_ins'] = df_cleaned['is_move_card_ins'].fillna(0)

# แปลงจาก float → int
df_cleaned['is_move_card_ins'] = df_cleaned['is_move_card_ins'].astype(int)

# แปลงจาก int → bool
df_cleaned['is_move_card_ins'] = df_cleaned['is_move_card_ins'].astype(bool)

# ตรวจสอบผลลัพธ์ก่อนส่ง
print(df_cleaned['is_move_card_ins'].dtype)
print(df_cleaned['is_move_card_ins'].unique())


bool
[ True False]


In [82]:
df_cleaned = df_cleaned.replace(r'^\s*$', np.nan, regex=True)


อาจจะมีปัญหา 
    |
    V

In [ ]:
df_cleaned.rename(columns={'agent_id': 'card_ins_id'}, inplace=True)


In [90]:
import pandas as pd
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

host = os.getenv('DB_HOST_test')
user = os.getenv('DB_USER_test')
password = os.getenv('DB_PASSWORD_test')
port = os.getenv('DB_PORT_test')
database = 'fininsurance'  

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')
rows_inserted = df_cleaned.to_sql('fact_card_ins', engine, if_exists='append', index=False)


In [87]:
# ตรวจสอบความยาวสูงสุดทุกคอลัมน์
for col in df_cleaned.columns:
    if df_cleaned[col].dtype == object:
        max_len = df_cleaned[col].astype(str).map(len).max()
        print(f"{col}: max length = {max_len}")


agent_id: max length = 30
agent_name: max length = 100
id_card: max length = 14
card_ins: max length = 4
card_ins_id: max length = 11
card_ins_type: max length = 14
card_ins_life: max length = 4
card_ins_type_life: max length = 4
file_card_ins_life: max length = 120
card_ins_life_exp: max length = 10
